In [ ]:
%pylab inline --no-import-all

import pandas as pd

pd.options.display.max_columns=50

## Load data

In [ ]:
# Data is assumed to be in directory data/
columns_names_dict = {"fecha_dato": "FETCH_DATE", "ncodpers": "CUST_ID", "ind_empleado": "EMPLYMT_STATUS", "pais_residencia": "RESID_CNTRY", "sexo": "CUST_SEX", "age": "CUST_AGE", "fecha_alta": "FSTHOLD_DATE", "ind_nuevo": "NEW_CUST", "antiguedad": "SENIORITY_MNTH", "indrel": "PRMRY_CUST", "ult_fec_cli_1t": "PRMRY_LASTDATE", "indrel_1mes": "CUST_TYPE", "tiprel_1mes": "CUST_ACTVTY", "indresi": "RESID_IDX", "indext": "FOREIGN_IDX", "conyuemp": "SPOUSE_EMP", "canal_entrada": "JOIN_CNL", "indfall": "DECEASED", "tipodom": "RESID_TYPE", "cod_prov": "RESID_ADD_CD", "nomprov": "RESID_ADD_NM", "ind_actividad_cliente": "CUST_ACTIVE", "renta": "CUST_HOUS_INCOM", "segmento": "CUST_SEGMNT", "ind_ahor_fin_ult1": "ACCNT_SAVIN", "ind_aval_fin_ult1": "ACCNT_GARNT", "ind_cco_fin_ult1": "ACCNT_CURNT", "ind_cder_fin_ult1": "ACCNT_DERIV", "ind_cno_fin_ult1": "ACCNT_PAYRL", "ind_ctju_fin_ult1": "ACCNT_JUNIO", "ind_ctma_fin_ult1": "ACCNT_MASPT", "ind_ctop_fin_ult1": "ACCNT_PARTI", "ind_ctpp_fin_ult1": "ACCNT_PARTP", "ind_deco_fin_ult1": "ACCNT_STDEP", "ind_deme_fin_ult1": "ACCNT_MTDEP", "ind_dela_fin_ult1": "ACCNT_LTDEP", "ind_ecue_fin_ult1": "ACCNT_EACCN", "ind_fond_fin_ult1": "ACCNT_FUNDS", "ind_hip_fin_ult1": "ACCNT_MORTG", "ind_plan_fin_ult1": "ACCNT_PENSN", "ind_pres_fin_ult1": "ACCNT_LOANS", "ind_reca_fin_ult1": "ACCNT_TAXES", "ind_tjcr_fin_ult1": "ACCNT_CCARD", "ind_valo_fin_ult1": "ACCNT_SECUR", "ind_viv_fin_ult1": "ACCNT_HOMEA", "ind_nomina_ult1": "PAYROLL", "ind_nom_pens_ult1": "PENSIONS", "ind_recibo_ult1": "DIRECTDEBT"}
data_train = pd.read_csv("data/train_ver2.csv", parse_dates=[0,6], nrows=10000)
data_train.rename(columns=columns_names_dict, inplace=True)
data_train.head()